In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
True
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph


In [2]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_teddynote.tools import GoogleNews
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_keyword])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()


In [5]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="messages")
# chunk_msg: 실시간 출력 메시지, metadata: 노드 정보
for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages"):

    # chatbot 노드에서 출력된 메시지만 출력
    if metadata["langgraph_node"] == "chatbot":
        if chunk_msg.content:
            print(chunk_msg.content, end="", flush=True)

    else:
        print(chunk_msg.content)
        print(f"\n\nmetadata: \n{metadata}\n\n")


- 오픈AI, '챗GPT'에 초기 AI 에이전트 기능 '태스크' 통합..."알림 설정·반복 요청 예약 가능" - AI타임스
- 오픈AI, '챗GPT'로 자동 소총 만들던 개발자 차단 - AI타임스
- "재미있게 바꿔줘" 명령했더니…AI의 엉뚱한 행동 - ZD넷 코리아
- 건강한 의심이 필요하다··· CIO가 답해야 할 AI 전략 질문 10가지 - CIO.com
- [ET시론] AI 디지털교과서, 오해 걷어내고 아이들의 미래 봐야 - 전자신문


metadata: 
{'langgraph_step': 2, 'langgraph_node': 'tools', 'langgraph_triggers': ['branch:chatbot:tools_condition:tools'], 'langgraph_path': ('__pregel_pull', 'tools'), 'langgraph_checkpoint_ns': 'tools:9523bece-cc18-e5a4-3fc8-3399f32345cd'}


다음은 AI 관련 최신 뉴스입니다:

1. **오픈AI, '챗GPT'에 초기 AI 에이전트 기능 '태스크' 통합**  
   - 사용자가 알림 설정 및 반복 요청 예약이 가능해졌습니다. (출처: AI타임스)

2. **오픈AI, '챗GPT'로 자동 소총 만들던 개발자 차단**  
   - 오픈AI가 위험한 사용을 방지하기 위한 조치를 취했습니다. (출처: AI타임스)

3. **"재미있게 바꿔줘" 명령했더니…AI의 엉뚱한 행동**  
   - AI의 예상치 못한 반응에 대한 기사입니다. (출처: ZD넷 코리아)

4. **건강한 의심이 필요하다··· CIO가 답해야 할 AI 전략 질문 10가지**  
   - CIO를 대상으로 한 AI 전략에 관한 질문 리스트입니다. (출처: CIO.com)

5. **[ET시론] AI 디지털교과서, 오해 걷어내고 아이들의 미래 봐야**  
   - AI 기반 디지털 교과서의 필요성과 미래에 대한 논의입니다. (출처: 전자신문)

더 궁금한 사항이 있으시면